In [ ]:
from IPython.display import display, HTML

display(
    HTML(
        """<style>
* {font-family:D2Coding;}
div.container{width:87% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input { font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:3px;}
table.dataframe{font-size:12px;}
</style>
"""
    )
)

In [ ]:
# 데이터 : 국토교통부 실거래가 공개시스템 https://rt.molit.go.kr/pt/xls/xls.do?mobileAt=

# <span style='color:red'>Ch1. 회귀모델저장</span>

In [3]:
import pandas as pd
import statsmodels.api as sm # 회귀코델
import joblib # 모델 저장 및 불러오기 (pkl 이나 joblib로 모델 저장, load)

df = pd.read_csv('../data/trade_apt_api.csv', comment='#')
df.info() 
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318 entries, 0 to 317
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   거래금액     318 non-null    int64  
 1   건축년도     318 non-null    int64  
 2   년        318 non-null    int64  
 3   법정동      318 non-null    object 
 4   아파트      318 non-null    object 
 5   월        318 non-null    int64  
 6   일        318 non-null    int64  
 7   전용면적     318 non-null    float64
 8   지번       318 non-null    object 
 9   지역코드     318 non-null    int64  
 10  층        318 non-null    int64  
 11  해제사유발생일  318 non-null    object 
 12  해제여부     318 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 32.4+ KB


거래금액       0
건축년도       0
년          0
법정동        0
아파트        0
월          0
일          0
전용면적       0
지번         0
지역코드       0
층          0
해제사유발생일    0
해제여부       0
dtype: int64

In [4]:
df.sample()

,거래금액,건축년도,년,법정동,아파트,월,일,전용면적,지번,지역코드,층,해제사유발생일,해제여부
278,65000,1993,2021,창신동,창신쌍용2,1,2,64.66,703,11110,14,-,-


In [14]:
# 독립변수
# pd.options.mode.copy_on_write = True # 데이터 복사 방지

X = df[['건축년도','전용면적','층']].copy()
X['const'] = 1

# 타켓변수
y = df['거래금액']

X.shape, y.shape


((318, 4), (318,))

In [18]:
model = sm.OLS(y, X).fit() # 회귀모델 
model.summary()

# R-squared (0.648) : X가 타켓(종속) 변수를 설명해 주는 수치 (설명도)
# Adj. R-squared (0.647) : 보정된 R-squared의 제곱 (설명도), 독립변수가 많아질수록 1에 가까워짐
# Durbin-Watson (1.352) : 자기상관성 여부 확인, 2에 가까울수록 자기상관성 없음
# 계수 : coef
# Skew (0.345) : 데이터의 분포가 종모양인지 확인, 0에 가까울수록 종모양
# Kurtosis (4.658) : 데이터의 분포가 종모양인지 확인, 3에 가까울수록 종모양
# 표준오차 : std err (표준오차가 작을수록 좋음)
# t-value : t (t-value가 클수록 유의미한 변수)
# P>|t| : 유의확률, 0.05 이하면 유의미한 변수
# [0.025 0.975] : 95% 신뢰구간


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   거래금액   R-squared:                       0.648
Model:                            OLS   Adj. R-squared:                  0.644
Method:                 Least Squares   F-statistic:                     192.4
Date:                Wed, 09 Jul 2025   Prob (F-statistic):           8.54e-71
Time:                        10:31:34   Log-Likelihood:                -3777.5
No. Observations:                 318   AIC:                             7563.
Df Residuals:                     314   BIC:                             7578.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
건축년도        1925.6916    212.616      9.057      0.000    1507.360    2344.023
전용면적         962.1507     47.367     20.313      0.000     868.955    1055.347
층           2058.1524    417.716      4.927      0.000    1236.276    2880.028
const      -3.855e+06   4.25e+05     -9.069      0.000   -4.69e+06   -3.02e+06
==============================================================================
Omnibus:                       20.985   Durbin-Watson:                   1.352
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               42.734
Skew:                           0.345   Prob(JB):                     5.25e-10
Kurtosis:                       4.658   Cond. No.                     4.33e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.33e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [19]:
X.iloc[0], y[0]

(건축년도     2002.00
 전용면적       84.82
 층           1.00
 const       1.00
 Name: 0, dtype: float64,
 80000)

In [ ]:
# 2차원으로
model.predict([[2005, 84, 8, 1]])[0] * 10000

1031843229.6999078

In [25]:
format(1031843229, ",")

'1,031,843,229'

In [30]:
# 모델 저장
joblib.dump(model, '../model/ex1_apt_price_regression.joblib')

['../model/ex1_apt_price_regression.joblib']

In [39]:
def predict_apt_price(year, square, floor):
    loaded_model = joblib.load("../model/ex1_apt_price_regression.joblib")
    # loaded_model 확인
    # print(loaded_model.summary())

    input_data = [[year, square, floor, 1]]
    result = round(loaded_model.predict(input_data)[0] * 10000, 1)
    return format(result, ",") + "원 입니다."


year = int(input("건축년도를 입력하세요: "))
square = int(input("전용면적(m²)을 입력하세요: "))
floor = int(input("층을 입력하세요: "))

predict_apt_price(year, square, floor)

'805,111,116.4원 입니다.'